# TFF/Anthos Hello World

Before running the notebook make sure that you have obtained access credentials to GCP by executing 

```
gcloud auth login
```

from the JupyterLab terminal

In [43]:
import collections
import time
import numpy as np
import grpc
import sys

import tensorflow as tf
import tensorflow_federated as tff

import nest_asyncio
nest_asyncio.apply()

### Define federated computation

Note that we need the explicit `tf.function` to make `tf.print` work.

In [44]:
@tff.tf_computation(tf.int64)
@tf.function
def hello(n):
    tf.print("Hello: ", n)
    return n

@tff.federated_computation(tff.FederatedType(tf.int64, tff.CLIENTS))
def compute_federated_sum(federated_n):
    return tff.federated_sum(tff.federated_map(hello, federated_n))

### Run computation locally

In [45]:
tff.backends.native.set_local_execution_context()

In [46]:
compute_federated_sum([1, 2, 1])

Hello:  2
Hello:  1
Hello:  1


4

### Run computation on the executor in the `server` cluster

In [47]:
port = 8000
ip_address = '10.108.9.176' # Cluster IP

channels = [grpc.insecure_channel(f'{ip_address}:{port}')]
tff.backends.native.set_remote_execution_context(channels, rpc_mode='REQUEST_REPLY')

In [48]:
compute_federated_sum([1])

1

#### Double check that the computation executed on the remote executor in the server cluster

Install `kubectl`

In [52]:
!gcloud components install kubectl

ERROR: (gcloud.components.install) 
You cannot perform this action because the Cloud SDK component manager 
is disabled for this installation. You can run the following command 
to achieve the same result for this installation: 

sudo apt-get install kubectl




In [50]:
server_cluster_name = 'tff-server'
zone = 'us-west1-a'

!gcloud container clusters get-credentials {server_cluster_name} --zone {zone}

client [kubectl]. To install, run
  $ gcloud components install kubectl

Fetching cluster endpoint and auth data.
kubeconfig entry generated for tff-server.


List pods

In [51]:
namespace = 'tff'

!kubectl get pods -n {namespace}

/bin/bash: kubectl: command not found


In [3]:
ip_address = '0.0.0.0'  #@param {type:"string"}
port = 8000  #@param {type:"integer"}

channels = [grpc.insecure_channel(f'{ip_address}:{port}') for _ in range(1)]

tff.backends.native.set_remote_execution_context(channels, rpc_mode='REQUEST_REPLY')

In [4]:
channels

In [5]:
#tff.backends.native.set_local_execution_context()

In [ ]:
compute_federated_sum([5])